# XFEL Wave Optics Simulation Tutorial - 

L.Samoylova <liubov.samoylova@xfel.eu>, A.Buzmakov <buzmakov@gmail.com>

Tutorial course on FEL Wavefront Propagation Simulations, 05/10/2016, SOS2016 Workshop, Trieste.

Version 28/11/2015

Wave optics software is based on SRW core library <https://github.com/ochubar/SRW>, available through WPG interactive framework <https://github.com/samoylv/WPG>

## Propagation through a beamline with CRLs: dispersion effect

### Import modules

In [ ]:
%matplotlib inline

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals


#Importing necessary modules:
import os
import sys
import copy
import time
import errno
import numpy as np
import pylab as plt

from IPython.display import Image

wpg_path = '/Users/lsamoylv/code/ghub/latest/WPG/'
#wpg_path = '/afs/desy.de/group/exfel/software/wpg/latest/'
sys.path.insert(0,wpg_path)

#Import base wavefront class
from wpg import Wavefront

#Gaussian beam generator
from wpg.generators import build_gauss_wavefront

#import srwl 
from wpg.srwlib import srwl

#import some helpers functions
from wpg.wpg_uti_exfl import calculate_theta_fwhm_cdr_s1
from wpg.wpg_uti_wf import calc_pulse_energy, averaged_intensity, calculate_fwhm, get_intensity_on_axis
from wpg.wpg_uti_wf import plot_wf, look_at_q_space, show_slices_hsv

plt.ion()

### define helper functions

In [ ]:
def mkdir_p(path):
    try:
        os.makedirs(path)
    except OSError as exc:  # Python >2.5
        if exc.errno == errno.EEXIST and os.path.isdir(path):
            pass
        else:
            raise
            


## define a beamline with offset mirror and CRL focusing to sample
### (day 1 layout SPB/SFX SASE1 instrument)

In [ ]:
from IPython.display import Image
Image(filename='figures/day1_1.png')

In [ ]:
%%file tutorial3_beamline_focusing_CRL_disp.py

def get_beamline(z):
    import os
    import wpg
    from wpg import Beamline
    from wpg.optical_elements import Aperture, Drift, CRL, Empty, Use_PP, Mirror_plane
    from wpg.wpg_uti_oe import show_transmission
    import numpy as np

    wpg_path = os.path.abspath(os.path.dirname(wpg.__file__))
    data_path = 'data_wpg_tutorial_03'
    data_common_path = 'data_common'
    bPrint = False

    # S1 beamline layout
    # Geometry ###
    src_to_hom1 = 257.8  # Distance source to HOM 1 [m]
    src_to_hom2 = 267.8  # Distance source to HOM 2 [m]
    src_to_crl = 887.8   # Distance source to CRL [m]
    src_to_exp = 920.803  #920.42 # Distance source to experiment [m]
    
    theta_om = 3.6e-3  # [rad]

    om_mirror_length = 0.8  # [m]
    om_clear_ap = om_mirror_length * theta_om

    # define the beamline:
    bl0 = Beamline()

    # Define HOM1.
    aperture_x_to_y_ratio = 1
    hom1 = Aperture(
        shape='r', ap_or_ob='a', Dx=om_clear_ap, Dy=om_clear_ap / aperture_x_to_y_ratio)

    # Define mirror profile
    # Apply distortions.
    mirrors_path = os.path.join(wpg_path, '..','samples', 'data_common')
    hom1_wavefront_distortion = Mirror_plane(orient='x', 
                                             theta=theta_om, 
                                             length=om_mirror_length, 
                                             range_xy=om_clear_ap/aperture_x_to_y_ratio, 
                                             filename=os.path.join(mirrors_path, 'mirror1.dat'), 
                                             scale=1, delim=' ')
    if bPrint:
        print('HOM1 WF distortion'); show_transmission(hom1_wavefront_distortion);

    # Free space propagation from hom1 to hom2
    hom1_to_hom2_drift = Drift(src_to_hom2 - src_to_hom1)

    # Define HOM2 as aperture.
    hom2 = Aperture('r', 'a', om_clear_ap, om_clear_ap / aperture_x_to_y_ratio)

    # drift to CRL aperture
    hom2_to_crl_drift = Drift(src_to_crl - src_to_hom2)

    # Define CRL
    crl_focussing_plane = 3  # Both horizontal and vertical.
    # Refractive index decrement (n = 1- delta - i*beta)
    #crl_delta = 4.7967e-6           #<=8.43 keV 4.7177e-06
    #crl_attenuation_length = 6.1e-3 #<=8.43 keV 6.3e-3    # Attenuation length [m], Henke data.
    be_data = np.loadtxt(os.path.join(data_common_path,'be_8430ev.txt'))
    crl_delta = be_data[:,1]  # Dummy numbers)
    crl_attenuation_length  = be_data[:,2] #Dummy numbers)
    crl_shape = 1         # Parabolic lenses
    crl_aperture = 5.0e-3  # [m]
    crl_curvature_radius = 5.8e-3  # [m]
    crl_number_of_lenses = 19
    crl_wall_thickness = 8.0e-5  # Thickness
    crl_center_horizontal_coordinate = 0.0
    crl_center_vertical_coordinate = 0.0
    crl_initial_photon_energy = 8.42e3  # [eV] ### OK ???
    crl_final_photon_energy = 8.44e3  # [eV]   ### OK ???

    crl = create_CRL(data_path,
                     'opd_crl_n{:d}_r_{:d}_e_{:d}_{:d}ev'.format(crl_number_of_lenses,int(crl_curvature_radius*1e6),
                                                           int(crl_initial_photon_energy),int(crl_final_photon_energy)),
                     _foc_plane=crl_focussing_plane,
                     _delta=crl_delta,
                     _atten_len=crl_attenuation_length,
                     _shape=crl_shape,
                     _apert_h=crl_aperture,
                     _apert_v=crl_aperture,
                     _r_min=crl_curvature_radius,
                     _n=crl_number_of_lenses,
                     _wall_thick=crl_wall_thickness,
                     _xc=crl_center_horizontal_coordinate,
                     _yc=crl_center_vertical_coordinate,
                     _void_cen_rad=None,
                     _e_start=crl_initial_photon_energy,
                     _e_fin=crl_final_photon_energy
                    )
    if bPrint:
        print('CRL'); show_transmission(crl)

    # drift to sample including offset z
    crl_to_sample = Drift(src_to_exp - src_to_crl + z)
    # Beamline:
    bl0.append(
        hom1, Use_PP(semi_analytical_treatment=0))
    zoom = 1.2
    bl0.append(hom1_wavefront_distortion,
               Use_PP(semi_analytical_treatment=0, zoom=zoom, sampling=zoom/0.8))
    bl0.append(hom1_to_hom2_drift, Use_PP(semi_analytical_treatment=0))
    zoom = 1.0
    bl0.append(hom2, Use_PP(semi_analytical_treatment=0,
                            zoom=zoom, sampling=zoom / 0.75))
    bl0.append(hom2_to_crl_drift, Use_PP(semi_analytical_treatment=1))
    zoom = 0.6
    #bl0.append(
    #    crl, Use_PP(semi_analytical_treatment=1, zoom=zoom, sampling=zoom/0.1))
    bl0.append(
        crl, Use_PP(semi_analytical_treatment=1, zoom=zoom, sampling=zoom/0.9))
    bl0.append(crl_to_sample, Use_PP(semi_analytical_treatment=1))

    return bl0

import sys
if sys.version_info[0] ==3:
    import pickle
else:
    import cPickle as pickle
def _save_object(obj, file_name):
    """
    Save any python object to file.
    
    :param: obj : - python objest to be saved
    :param: file_name : - output file, wil be overwrite if exists
    """
    with open(file_name,'wb') as f:
        pickle.dump(obj, f)

def _load_object(file_name):
    """
    Save any python object to file.
    
    :param: file_name : - output file, wil be overwrite if exists
    :return: obj : - loaded pthon object
    """
    res = None
    with open(file_name,'rb') as f:
        res = pickle.load(f)
        
    return res

def create_CRL(directory,file_name,**args):
    """
    This function build CLR or load it from file.
    Out/input filename builded as sequence of function parameters.
    Adiitinal parameters (*args) passed to srwlib.srwl_opt_setup_CRL function
    
    :param directory: output directory
    :param file_name: CRL file name
    :return: SRWL CRL object
    """
        
    import os
    from wpg.optical_elements import CRL

    full_path = os.path.join(directory, file_name+'.pkl')
    
    if  os.path.isfile(full_path):
        print('Found file {}. CLR will be loaded from file'.format(full_path))
        res = _load_object(full_path)
        return res
    else:
        print('CLR file NOT found. CLR will be recalculated and saved in file {}'.format(full_path))
        #res = srwl_opt_setup_CRL(*args)
        #print('{:s},{:s},{:s},{:s},{:s},{:s},{:s},{:s},{:s},{:s},{:s},{:s},{:s}'.format(*args))
        res = CRL(**args)
        mkdir_p(os.path.dirname(full_path))
        _save_object(res, full_path)
        return res 

def mkdir_p(path):
    import os
    import errno
    try:
        os.makedirs(path)
    except OSError as exc:  # Python >2.5
        if exc.errno == errno.EEXIST and os.path.isdir(path):
            pass
        else:
            raise


### define source 

In [ ]:
data_path = 'data_wpg_tutorial_03';mkdir_p(data_path)
out_prefix = 'gb_200as_'
src_oe1 = 257.8 # Distance to the first OE [m]
fname = '{0:s}at_{1:.0f}_m'.format(out_prefix,src_oe1);
bname = fname

# Central photon energy.
ekev = 8.43 # Energy [keV]

# Pulse parameters.
qnC = 0.1               # e-bunch charge, [nC]
pulse_duration = 9.e-15 # [s] <-is not used really, only ~coh time pulse duration has physical meaning 
pulseEnergy = 1.5e-3    # total pulse energy, J
coh_time = 0.2e-15      # [s]<-should be SASE coherence time, then spectrum will be the same as for SASE 
                        # check coherence time for 8 keV 0.5 nC SASE1

# Angular distribution
theta_fwhm = calculate_theta_fwhm_cdr_s1(ekev,qnC) # CDR2011 empirical formula
theta_fwhm = 3.2e-6 #empiric value for nzc 15 (linear SASE before saturation)
print('divergence FWHM {0:.1f} urad'.format(theta_fwhm*1e6))

In [ ]:
# Gaussian beam parameters
wlambda = 12.4*1e-10/ekev        # wavelength 
w0 = wlambda/(np.pi*theta_fwhm)  # FWHM beam waist; 
w0_a = w0/(2*np.sqrt(np.log(2))) # sigma of amplitude
zR = (np.pi*w0_a**2)/wlambda     # Rayleigh length, max WF curvature

# expected beam radius at OE1 position to get the range of the wavefront 
print('Intensity waist size: {:.0f} um FWHM'.format(w0*1e6))
print('Rayleigh length: {:.1f} m'.format(zR))
print('expected FWHM at distance {:.1f} m: {:.2f} mm'.format(src_oe1,theta_fwhm*src_oe1*1e3))

In [ ]:
bSaved=True
sig_num = 5.5
num_points = 400 #number of points
range_xy = w0_a * np.sqrt(1+(src_oe1/zR)**2)/np.sqrt(2)*sig_num;
print('range_xy at OE1: {:.1f} mm'.format(range_xy*1e3))
dx = 10.e-6; range_xy = dx*(num_points-1);
print('range_xy at OE1: {:.1f} mm'.format(range_xy*1e3))
nslices = 20; 

### create Gaussian beam

In [ ]:
srwl_wf = build_gauss_wavefront(num_points, num_points, nslices, ekev, -range_xy/2, range_xy/2,
                                -range_xy/2, range_xy/2 ,coh_time/np.sqrt(2), 
                                w0_a, w0_a, src_oe1,
                                pulseEn=pulseEnergy, pulseRange=8.)
wf = Wavefront(srwl_wf)
z0 = src_oe1
#defining name HDF5 file for storing wavefront
#store wavefront to HDF5 file 
if bSaved:     
    wf.store_hdf5(os.path.join(data_path,fname+'.h5')); 
    print('saving WF to {:s}'.format(os.path.join(data_path,fname+'.h5')))

xx=calculate_fwhm(wf);
print('FWHM at distance {:.1f} m: {:.2f} x {:.2f} mm2'.format(z0,xx[u'fwhm_x']*1e3,xx[u'fwhm_y']*1e3));

### show Gaussian wavefront in R- and Q-space

In [ ]:
plot_wf(wf)
look_at_q_space(wf)

### load FEL SASE pulse 
###  _copy FEL pulse from tutorial1 data folder_
### _change formatting from raw to code to load FEL beam_

### propagate through the beamline

In [ ]:
#loading beamline from file
import imp
custom_beamline = imp.load_source('custom_beamline', 'tutorial3_beamline_focusing_CRL_disp.py')
get_beamline = custom_beamline.get_beamline
#za = np.array([-30.e-3,-25.e-3,-20e-3,-15.e-3,-10.e-3])
za = np.array([0.])
fwhm_x = np.zeros(len(za), dtype='float32')
fwhm_y = np.zeros(len(za), dtype='float32')
for idx in range (len(za)):
    z = za[idx]
    bl = get_beamline(z)
    if idx==0: print(bl)
    wf=Wavefront() ;wf.load_hdf5(os.path.join(data_path,fname+'.h5'))
    srwl.ResizeElecField(wf._srwl_wf, 't', [0, 5, 1]);
    print('switching in frequency domain...');t0 = time.time();
    srwl.SetRepresElecField(wf._srwl_wf, 'f') # <---- switch to frequency domain
    print('done in {:.2f} s'.format(time.time()-t0))
    print('decreasing range in frequency domain...');t0 = time.time();
    srwl.ResizeElecField(wf._srwl_wf, 'f', [0, 0.33, 1.]);print('done in {:.2f} s'.format(time.time()-t0))
    plot_wf(wf)
    print('propagating WF...');t0 = time.time();
    bl.propagate(wf); print('in {:.2f} s'.format(time.time()-t0))
    plot_wf(wf)
    print('increasing range in frequency domain...');t0 = time.time();
    srwl.ResizeElecField(wf._srwl_wf, 'f', [0, 3., 1.]); print(' done in {:.2f} s'.format(time.time()-t0))
    print('switching in time domain...');t0 = time.time();
    srwl.SetRepresElecField(wf._srwl_wf, 't'); print('done in {:.2f} s'.format(time.time()-t0))
    print('resizing after propagation...');t0 = time.time();
    srwl.ResizeElecField(wf._srwl_wf, 't', [0, 0.6, 1]); print('done in {:.2f} s'.format(time.time()-t0))
    #plot_wf(wf)
    xx=calculate_fwhm(wf);
    print('FWHM at {:.1f} mm offset: {:.2f} x {:.2f} um2'.format(z*1e3, xx[u'fwhm_x']*1e6,xx[u'fwhm_y']*1e6));
    fwhm_x[idx] = xx[u'fwhm_x'];fwhm_y[idx] = xx[u'fwhm_y'];
    #plot_wf(wf)

### show propagated wavefront

In [ ]:
xx=calculate_fwhm(wf);
print('FWHM propagated beam: {:.2f} x {:.2f} um2'.format(xx[u'fwhm_x']*1e6,xx[u'fwhm_y']*1e6));
plot_wf(wf)
look_at_q_space(wf)    

### save propagated pulse 

### load FEL SASE pulse 
**copy FEL pulse**   `s1_8_43_100_xy75_ss10_nzc15_1_0000001.h5`   from **tutorial1 data folder**

In [ ]:
wf=Wavefront()
bname = 's1_8_43_100_xy75_ss10_nzc15_1_0000001'
fname = bname+'_prop_at_256m'
wf.load_hdf5(os.path.join(data_path,fname+'.h5'))
#plot_wf(wf)
#look_at_q_space(wf)

### show initial FEL SASE wavefront

from wpg.srwlib import srwl
#Resizing: decreasing Range of Horizontal and Vertical Position:
#srwl.ResizeElecField(wf._srwl_wf, 'c', [0, 0.48, 0.48/0.16, 0.48,  0.48/0.16]);
xx=calculate_fwhm(wf);
print('source size FWHM: {:.3f} x {:.3f} mm2'.format(xx[u'fwhm_x']*1e3,xx[u'fwhm_y']*1e3));
plot_wf(wf)
look_at_q_space(wf)    
#wpg.srwlib.srwl.SetRepresElecField(wf._srwl_wf, 'f')

### propagate through the beamline

#loading beamline from file
import imp
custom_beamline = imp.load_source('custom_beamline', 'tutorial2_beamline_mirror.py')
get_beamline = custom_beamline.get_beamline
bl = get_beamline()
print(bl)

wf=Wavefront() ;wf.load_hdf5(os.path.join(data_path,fname+'.h5'))
srwl.SetRepresElecField(wf._srwl_wf, 'f') # <---- switch to frequency domain
print('propagating through beamline...');t0 = time.time();
bl.propagate(wf)
print('done in {:.2f} s'.format(time.time()-t0))
srwl.SetRepresElecField(wf._srwl_wf, 't')


### show propagated wavefront

xx=calculate_fwhm(wf);
print('FWHM propagated beam: {:.2f} x {:.2f} mm2'.format(xx[u'fwhm_x']*1e3,xx[u'fwhm_y']*1e3));
plot_wf(wf)
look_at_q_space(wf)    